# Homework Week 2

In [1]:
import warnings
warnings.filterwarnings('ignore')

## Q1. Install MLflow

In [2]:
import mlflow

In [3]:
version = !mlflow --version
print(f"The version of MLflow is {version.fields(2)}")

The version of MLflow is ['1.26.1']


## Q2. Download and preprocess the data

In [4]:
!python preprocess_data.py --raw_data_path ~/notebooks/mlops-zoomcamp/week2/data/ --dest_path ./output

In [5]:
out = !ls ./output
print(out)
print(f"In total there are {len(out)} files created in the output.")

['dv.pkl', 'test.pkl', 'train.pkl', 'valid.pkl']
In total there are 4 files created in the output.


# Q3. Train a model with autolog

In [6]:
!python train.py

2022/05/31 20:17:10 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/ubuntu/anaconda3/envs/exp-tracking-env-week2/lib/python3.9/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils."


In [7]:
# connect to client to retrieve number of parameter logged
from mlflow.tracking import MlflowClient
TRACKING_URI = "sqlite:///mlflow.db"
client = MlflowClient(tracking_uri = TRACKING_URI )

client.list_experiments()

[<Experiment: artifact_location='./mlruns/0', experiment_id='0', lifecycle_stage='active', name='Default', tags={}>,
 <Experiment: artifact_location='./mlruns/2', experiment_id='2', lifecycle_stage='active', name='nyc-experiment-homework-w2', tags={}>]

In [8]:
client.list_run_infos(experiment_id=2)

[<RunInfo: artifact_uri='./mlruns/2/8bb6930f50334314bd83bdcfe81f96cf/artifacts', end_time=1654028230543, experiment_id='2', lifecycle_stage='active', run_id='8bb6930f50334314bd83bdcfe81f96cf', run_uuid='8bb6930f50334314bd83bdcfe81f96cf', start_time=1654028204827, status='FINISHED', user_id='ubuntu'>,
 <RunInfo: artifact_uri='./mlruns/2/080d9b0c64a248acb0b281867e58dc51/artifacts', end_time=1654007723100, experiment_id='2', lifecycle_stage='active', run_id='080d9b0c64a248acb0b281867e58dc51', run_uuid='080d9b0c64a248acb0b281867e58dc51', start_time=1654007697579, status='FINISHED', user_id='ubuntu'>,
 <RunInfo: artifact_uri='./mlruns/2/119cfd7c3ebd439088a8f158288d3c07/artifacts', end_time=1654006259855, experiment_id='2', lifecycle_stage='active', run_id='119cfd7c3ebd439088a8f158288d3c07', run_uuid='119cfd7c3ebd439088a8f158288d3c07', start_time=1654006234336, status='FINISHED', user_id='ubuntu'>,
 <RunInfo: artifact_uri='./mlruns/2/51e5e76aca494759b8f5c90988392f13/artifacts', end_time=1654

In [9]:
run = client.get_run(run_id ='4537467a7cf448a4af17ab0ac478d4e7' )

params = run.data.params
params

{'bootstrap': 'True',
 'ccp_alpha': '0.0',
 'criterion': 'squared_error',
 'max_depth': '10',
 'max_features': '1.0',
 'max_leaf_nodes': 'None',
 'max_samples': 'None',
 'min_impurity_decrease': '0.0',
 'min_samples_leaf': '1',
 'min_samples_split': '2',
 'min_weight_fraction_leaf': '0.0',
 'n_estimators': '100',
 'n_jobs': 'None',
 'oob_score': 'False',
 'random_state': '0',
 'verbose': '0',
 'warm_start': 'False'}

In [10]:
print(f"The number of parameters automatically logged is {len(params)}.")

The number of parameters automatically logged is 17.


# Q4. Launch the tracking server locally

In [11]:
print("In addition to backend-store-uri we need also to pass the artifacts-destination in order to configure properly the server.")

In addition to backend-store-uri we need also to pass the artifacts-destination in order to configure properly the server.


# Q5. Tune the hyperparameters of the model

In [12]:
!python hpo.py

Traceback (most recent call last):
  File "/home/ubuntu/notebooks/mlops-zoomcamp/week2/homework/hpo.py", line 13, in <module>
    mlflow.set_experiment("random-forest-hyperopt")
  File "/home/ubuntu/anaconda3/envs/exp-tracking-env-week2/lib/python3.9/site-packages/mlflow/tracking/fluent.py", line 130, in set_experiment
    raise MlflowException(
mlflow.exceptions.MlflowException: Cannot set a deleted experiment 'random-forest-hyperopt' as the active experiment. You can restore the experiment, or permanently delete the experiment to create a new one.


In [13]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids= 3,
    filter_string="",
    run_view_type= ViewType.ACTIVE_ONLY,
    max_results= 6,
    order_by= ["metrics.rmse ASC"]
)

for run in runs:
    print(f"run id : {run.info.run_id}, rmse : {run.data.metrics['rmse']:.3f}")


run id : 7aed9597b38640baa992a1d09c52db9c, rmse : 6.628
run id : 5474545d1a1d4a30b97bc6e02ed958a2, rmse : 6.628
run id : 340d2655e5d74a73bf3cb0d2f3d3507c, rmse : 6.628
run id : 56c4c8a390074e7fb6b768bd653219ec, rmse : 6.628
run id : ba927dfb77724e3f9a5b95b07ea32bf8, rmse : 6.630
run id : 86973faba61d4dc9868ac00272e29737, rmse : 6.630


In [14]:
print(f"The best validation RMSE is {runs[0].data.metrics['rmse']:.3f}.")

The best validation RMSE is 6.628.


# Q6. Promote the best model to the model registry

In [15]:
!python register_model.py 

Traceback (most recent call last):
  File "/home/ubuntu/notebooks/mlops-zoomcamp/week2/homework/register_model.py", line 17, in <module>
    mlflow.set_experiment(EXPERIMENT_NAME)
  File "/home/ubuntu/anaconda3/envs/exp-tracking-env-week2/lib/python3.9/site-packages/mlflow/tracking/fluent.py", line 130, in set_experiment
    raise MlflowException(
mlflow.exceptions.MlflowException: Cannot set a deleted experiment 'random-forest-best-models' as the active experiment. You can restore the experiment, or permanently delete the experiment to create a new one.


In [16]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids= 4,
    filter_string="",
    run_view_type= ViewType.ACTIVE_ONLY,
    max_results= 6,
    order_by= ["metrics.test_rmse ASC"]
)

for run in runs:
    print(f"run id : {run.info.run_id}, rmse : {run.data.metrics['test_rmse']:.3f}")


run id : 388e33dce80f4a1fa2cbdbfa9c06cd49, rmse : 6.548
run id : 168ddcf6b3c44035a2a18b2111c359e8, rmse : 6.548
run id : eb044b7640034b67a0de17b248ee8bec, rmse : 6.548
run id : 906c6af39c09487ab3f720e08a3d10c0, rmse : 6.548
run id : d92c174a9a8e4440b885bf4e16876fa4, rmse : 6.548
run id : 13398b8d56fb492383fa87e488a5fdcc, rmse : 6.548


In [17]:
print(f"The best test RMSE is {runs[0].data.metrics['test_rmse']:.2f}.")

The best test RMSE is 6.55.
